# Question 1

Write a function to get the last item in a complete tree. This is easy to do if the complete tree were implemented using arrays. How would we do this if the tree was implemented using nodes?

In [1]:
def get_last_array(tree:list):
    if type(tree) == list:
        return tree[-1]
    else:
        raise TypeError("Input type must be a list")   

In [2]:
from dsa.tree import Node, Tree
from dsa.queue import Queue

root = Node(1)
root.left = Node(2)
root.right = Node(3)
root.left.left = Node(4)
root.left.right = Node(5)
root.right.left = Node(6)
root.right.right = Node(7)
root.left.left.left = Node(8)
root.left.left.right = Node(9)
root.left.right.left = Node(10)
root.left.right.right = Node(11)
root.right.left.left = Node(12)
root.right.left.right = Node(13)
root.right.right.left = Node(14)
root.right.right.right = Node(15)

tree = Tree(root)


tree.print()

         15
      7
         14
   3
         13
      6
         12
1
         11
      5
         10
   2
         9
      4
         8


In [13]:
q =Queue()
q.enqueue(tree.root)

a = q.dequeue()

a.value


1

In [26]:
def find_last_node(tree:Tree):
    
    root = tree.root
    
    if root.value == None:
        return None
    
    q = Queue()
    q.enqueue(root)

    last_node = q.peek()

    while q.is_empty() == False:
        last_node = q.dequeue()
        if last_node.left:
            q.enqueue(last_node.left)
        if last_node.right:
            q.enqueue(last_node.right)
        
    return last_node.value

find_last_node(tree)
        

15

The above functions return the final item in a complete binary tree. The array version simply returns the final index of an array, which uses array indexing so uses $O(1)$ time and $O(1)$ space, since no new data is being created or stored.

The tree version works by implementing level order traversal. We cannot simply traverse down the left/right to find the last level and then search it because in a tree, values are referenced using nodes, meaning there's no way to go left or right in a tree, you can only go up/down. Thus, we cleverly leverage queues to enqueue and dequeue children and update our last seen node. We enqueue the root when initializing the algorithm. 

We then while loop through the tree, deuqueing the current node. The current node's left and right children are then added to the back of the queue. The next item in the dequeued will be in the same level as the parent of the child nodes just enqueued, because the tree is complete, and this process continues until a level has been exhausted. Since we are traversing each level from left to right and the tree is guaranteed to be complete, the last node visited will be the final value, and the last node is updated to be the current node everytime a new node is dequeued. This algorithm has a time complexity of $O(n)$ since we are visiting every node once. It uses at most $\frac{n}{2}$ nodes in the queue, which means that overall the algorithm uses $O(n)$ space.

# Question 2

Write a function that accepts a string and returns an array of the characters in the string sorted by frequency (from most frequent to least frequent).

You must use a heap to sort the characters. You can use the heap in the dsa package, Python's heapq package or write your own. 

Example: 
The input

```python
"open sesame"
```

should return:
```python
['e', 's', 'p', 'o', 'n', 'm', 'a', ' ']

```

Note: characters that have the same frequency can appear in any order

In [6]:
from collections import defaultdict
from dsa.heap import Heap
def sort_char_by_freq(input:str) -> list[str]:
    freq = defaultdict(int)
    
    for char in input:
        freq[char] += 1
    
    freq2 = defaultdict(list)
    for char, frequency in freq.items():
        freq2[frequency].append(char)

    heap = Heap(maxheap=True)

    for key in freq2.keys():
        heap.insert(key)

    chars = []
    
    while heap.count() > 0:
        current_freq = heap.pop()
        for char in freq2[current_freq]:
            chars.append(char)
    
    return chars

sort_char_by_freq("open sesame")

['e', 's', 'o', 'p', 'n', ' ', 'a', 'm']

This function leverages the heap condition to parse a string, and return the characters back to the user ordered by their frequency in the input string. First, we iterate through the input string and create a hashmap where each character in the string is a key, and the value is the count of the character. We use a defaultdict instead of a normal dictionary since we cannot initialize a value for a key that doesn't already exist in a dictionary when for looping through the string.

Next, we create a second dictionary which is essentially the reverse of the first one, where the count is the key and the character is the value. To reduce the overall number of computations, the hashmap values are actually lists containing all values that have that count.

Next, we initialize a max heap and store all of the keys from the second hash map (the counts) into the heap. We then pop the top value from the heap until it is empty, and use that value to retrieve the characters from the hash map of counts, add them to a list. Since the top of the max heap is always the maximum value, we know that the value corresponding to that key is the next frequency.

This code has a time complexity of $O(nlogn)$. All other parts are $O(n)$ except the while loop. The loop runs at most n times (in case each character in the input string appears once) and retrieving an element from a heap is $O(1)$, but heapifying down after popping is $O(logn)$ worst case. This code uses 2 hash maps and 1 heap, all of which use $O(n)$ space, meaning the overall space complexity is $O(n)$.





# Question 3

Write a function that accepts an array of words and returns the longest common prefix. Write it so that it performs efficiently.

For example, given the array
```
words = ["apple", "appetite", "apparatus", "appliance"]
```

The function should return

```
"app"
```

In [21]:
from dsa.trie import Trie

def longest_common_prefix(words:list[str]) -> str:
    if not words:
        return ""
    
    trie = Trie()

    for word in words:
        trie.insert(word)

    node = trie.root
    prefix = []

    while node and len(node.children) == 1 and not trie.end_char:
        char, next_node = next(iter(node.children.items()))
        prefix.append(char)

        node = next_node

    return ''.join(prefix)

    

longest_common_prefix(["apple", "appetite", "apparatus", "appliance"])


''

In [23]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_end_of_word = True

def longest_common_prefix(words):
    if not words:
        return ""
    
    trie = Trie()
    for word in words:
        trie.insert(word)
    
    node = trie.root
    prefix = []
    while node and len(node.children) == 1 and not node.is_end_of_word:
        char, next_node = next(iter(node.children.items()))
        prefix.append(char)
        node = next_node
    return ''.join(prefix)

# Example usage
words = ["apple", "appetite", "apparatus", "appliance"]
print(longest_common_prefix(words))

app


# Question 4

Write a function that accepts an array of words and then returns the shortest unique prefix of each word. 
For example:

`words = ['apple', 'banana', 'cherry', 'cranberry', 'grape', 'grapefruit']`

```python
words = ['apple', 'banana', 'cherry', 'cranberry', 'grape', 'grapefruit'] 

# 'apple' returns 'a'
# 'banana' returns 'b'
# 'cherry' returns 'ch'
# 'cranberry' returns 'cr'
# 'grape' returns 'grape'
# 'grapefruit' returns 'grapef'
# returns:
['a', 'b', 'ch', 'cr', 'grape', 'grapef']
```